In [1]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import os, codecs
%matplotlib inline
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn import metrics as skmetrics

2017-06-09 16:34:29,332 : INFO : 'pattern' package found; tag filters are available for English
/usr/lib64/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Tweets 50 / 300 /skip gram

In [2]:
model = gensim.models.Word2Vec.load('/home/bahbbc/workspace/masters-big5/models/tweet50-300-skip.model')
model.init_sims(replace=True)

2017-06-09 16:34:30,700 : INFO : loading Word2Vec object from /home/bahbbc/workspace/masters-big5/models/tweet50-300-skip.model
2017-06-09 16:34:37,173 : INFO : loading wv recursively from /home/bahbbc/workspace/masters-big5/models/tweet50-300-skip.model.wv.* with mmap=None
2017-06-09 16:34:37,174 : INFO : loading syn0 from /home/bahbbc/workspace/masters-big5/models/tweet50-300-skip.model.wv.syn0.npy with mmap=None
2017-06-09 16:34:41,562 : INFO : setting ignored attribute syn0norm to None
2017-06-09 16:34:41,564 : INFO : loading syn1neg from /home/bahbbc/workspace/masters-big5/models/tweet50-300-skip.model.syn1neg.npy with mmap=None
2017-06-09 16:34:45,652 : INFO : setting ignored attribute cum_table to None
2017-06-09 16:34:45,665 : INFO : loaded /home/bahbbc/workspace/masters-big5/models/tweet50-300-skip.model
2017-06-09 16:34:49,260 : INFO : precomputing L2-norms of word weight vectors


In [3]:
num_features= 300

In [4]:
df = pd.read_csv('/home/bahbbc/personality-normalized-word2vec.csv', encoding='utf-8')
df.shape

(1039, 186)

In [5]:
df.openness_m.value_counts()

1    543
0    496
Name: openness_m, dtype: int64

In [6]:
train_w2v_data, test_w2v_data = train_test_split(df, test_size=0.3, random_state=42)

In [7]:
def predict(vectorizer, classifier, data):
    data_features = vectorizer.transform(data['formatted_text'])
    predictions = classifier.predict(data_features)
    target = data['openness_m']
    evaluate_prediction(predictions, target)

In [8]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.wv.vocab:
            mean.append(wv.wv.syn0norm[wv.wv.vocab[word].index])
            all_words.add(wv.wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(num_features,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list ])

In [9]:
def w2v_tokenize_text(text):
    tokens = []
    if text is np.nan:
        return []
    for sent in nltk.sent_tokenize(text, language='portuguese'):
        for word in nltk.word_tokenize(sent, language='portuguese'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [10]:
test_tokenized = test_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values
train_tokenized = train_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values

In [11]:
%%time
X_train_word_average = word_averaging_list(model,train_tokenized)
X_test_word_average = word_averaging_list(model,test_tokenized)

2017-06-09 16:36:31,303 : WARNING : cannot compute similarity with no input []
2017-06-09 16:36:31,630 : WARNING : cannot compute similarity with no input []
2017-06-09 16:36:31,638 : WARNING : cannot compute similarity with no input []
2017-06-09 16:36:32,275 : WARNING : cannot compute similarity with no input []
2017-06-09 16:36:32,528 : WARNING : cannot compute similarity with no input []
2017-06-09 16:36:32,904 : WARNING : cannot compute similarity with no input []
2017-06-09 16:36:33,618 : WARNING : cannot compute similarity with no input []
2017-06-09 16:36:33,709 : WARNING : cannot compute similarity with no input []
2017-06-09 16:36:33,944 : WARNING : cannot compute similarity with no input []
2017-06-09 16:36:34,102 : WARNING : cannot compute similarity with no input []
2017-06-09 16:36:34,161 : WARNING : cannot compute similarity with no input []


CPU times: user 3.87 s, sys: 24.1 ms, total: 3.89 s
Wall time: 3.89 s


In [12]:
del model

In [13]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

force = False
model_trainer = RandomizedSearchCV(
    n_iter=1, 
    estimator=RandomForestClassifier(),
    param_distributions={
        "criterion": ["gini"],
        "n_estimators": [10],
        "max_features": ["log2"],
        "max_depth": [None],
        "bootstrap": [True],
        "oob_score": [True],
        "class_weight": ["balanced"],
        "random_state": [42]
    },
    scoring="f1",
    verbose=True,
    refit=True,
    cv=10,
    n_jobs=-1
)

In [14]:
%%time
model_trainer.fit(X_train_word_average, train_w2v_data['openness_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


/usr/lib64/python2.7/site-packages/sklearn/ensemble/forest.py:439: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/usr/lib64/python2.7/site-packages/sklearn/ensemble/forest.py:439: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/usr/lib64/python2.7/site-packages/sklearn/ensemble/forest.py:444: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/usr/lib64/python2.7/site-packages/sklearn/ensemble/forest.py:444: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/usr/lib64/python2.7/site-packages/sklearn/ensemble/forest.py:439: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute a

CPU times: user 232 ms, sys: 64.8 ms, total: 297 ms
Wall time: 663 ms


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.4s finished
/usr/lib64/python2.7/site-packages/sklearn/ensemble/forest.py:439: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/usr/lib64/python2.7/site-packages/sklearn/ensemble/forest.py:444: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


In [15]:
yp = model.predict(X_test_word_average)
yt = test_w2v_data['openness_m']

In [16]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,96,40
1,95,81


In [17]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.50      0.71      0.59       136
          1       0.67      0.46      0.55       176

avg / total       0.60      0.57      0.56       312



In [18]:
accuracy_score(yt, yp)

0.56730769230769229

In [19]:
print '--------- TRAIN -----------------'

--------- TRAIN -----------------


In [20]:
ytp = model.predict(X_train_word_average)
ytt = train_w2v_data['openness_m']

In [21]:
print skmetrics.classification_report(y_true=ytt, y_pred=ytp)

             precision    recall  f1-score   support

          0       0.96      0.99      0.98       360
          1       0.99      0.96      0.98       367

avg / total       0.98      0.98      0.98       727



In [22]:
print accuracy_score(ytt, ytp)

0.977991746905
